네이버 시가총액 (코스피500개, 코스닥300개)

In [2]:
import csv
import requests
import datetime
from bs4 import BeautifulSoup

def stock_data(columns, sosok):
    data = list()
    data = data + [sosok]
    code = list()    
    #fn_link = list()
    for column in columns:        
        data = data + [column.text.strip()]            
        a = column.find("a",{"class":"tltle"})

        if a:
            code = [a.attrs["href"][20:]]
            #code = ["https://finance.naver.com" + a.attrs["href"]] 
            code_str = str(code)[4:10]
            #fn_link = "http://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?pGB=1&gicode=A"+code_str+"&cID=&MenuYn=Y&ReportGB=&NewMenuID=11&stkGb=701"
            fn_link = "https://comp.fnguide.com/SVO/WooriRenewal/Snapshot.asp?pGB=12&gicode=A"+code_str
            #print(fn_link)
            data = data + code + [fn_link]

    return data[0:-1]



today = datetime.date.today().strftime('%y%m%d')
filename = today + "_sise_code.csv"

f = open(filename, "w", encoding="euc-kr", newline="")
writer = csv.writer(f)

title = "구분,No,종목명,종목코드,fn링크,현재가,전일비,등락률,액면가,시가총액,상장주식수,외국인비율,거래량,PER,ROE".split(",")
print(type(title))
writer.writerow(title)

for sosok in range(2):
    if sosok==0:
        last_page = 11
    else:
        last_page = 7    

    url = "https://finance.naver.com/sise/sise_market_sum.nhn?sosok=" + str(sosok) + "&page="

    for page in range(1, last_page):
        res =requests.get(url + str(page))
        res.raise_for_status()
        soup = BeautifulSoup(res.text, "lxml")

        data_rows = soup.find("table", attrs={"class":"type_2"}).find("tbody").find_all("tr")
        for row in data_rows:
            columns = row.find_all("td")
            #print(columns)
            #print("*****")
            if len(columns) <= 1:
                continue        
            #data = [column.get_text().strip() for column in columns]
            #print(data)
            #writer.writerow(data)
            
            data = stock_data(columns, sosok)
            #print(type(data))
            #print(data)

            writer.writerow(data)

print('save')
f.close()


<class 'list'>
save


관심종목 등락률 조회 / 관심종목에 종목코드 값 추가

In [ ]:
import csv
import pandas as pd
import re
import datetime

pd.set_option('display.max_rows', None)

today = datetime.date.today().strftime('%y%m%d')
filename = today + "_sise_code.csv"

df = pd.read_csv(filename, encoding="euc-kr")
df_interest = pd.read_csv("interest.csv", encoding="euc-kr")

df['등락률2']=df['등락률'].str.replace('%','').astype(float)
df['전일비2']=df['전일비'].str.replace(r'\s+',' ', regex=True)

df_stock = df.loc[:, ['구분','종목명','종목코드','현재가','전일비2','등락률2','외국인비율','PER','ROE']]
df_stock_updown_sort = df_stock.sort_values(by=['등락률2','현재가'], ascending=True)
# df_stock_siga_sort = df_stock.sort_values(by=['시가총액'])

#df_pr_ch = df3.loc[((df3['등락률2'] > 10) | (df3['등락률2'] < -5)) & (df3['외국인비율'] > 5), ['구분','종목명','현재가','전일비2','등락률2','외국인비율','PER','ROE']]
#display(df_pr_ch)   

interest_list = df_interest['종목명'].tolist()
df_sname =df_stock.loc[df_stock['종목명'].isin(interest_list)]                                                                                                                                                                            
display(df_sname)

f = open("interest.csv", "w", encoding="euc-kr", newline="")
df_interest_update = df_sname.loc[:,['종목명','종목코드']]
df_interest_update.to_csv(f)

f.close()

print('관심종목 종목코드 입력완료')



관심종목 실시간 조회

In [ ]:
import csv
import requests
import pandas as pd
from bs4 import BeautifulSoup

def naver_data(code):
    url = "https://finance.naver.com/item/main.nhn?code=" + code    
    res =requests.get(url)
    res.raise_for_status()
    soup = BeautifulSoup(res.text, "html.parser")

    try :
        s_name = soup.find('div',{'class':'h_company'}).find('h2').text.replace(u'\xa0','')
        s_price = soup.find('p',{'class':'no_today'}).find('span',{'class':'blind'}).text.replace(u'\xa0','')
        s_ch = soup.find('p',{'class':'no_exday'}).find_all('em')[0].find('span',{'class':'blind'}).text.replace(u'\xa0','')
        s_ch1 = soup.find('p',{'class':'no_exday'}).find_all('em')[1].find('span',{'class':'ico'}).text.replace(u'\xa0','')
        s_ch2 = soup.find('p',{'class':'no_exday'}).find_all('em')[1].find('span',{'class':'blind'}).text.replace(u'\xa0','')
        #s_pr1 = soup.find('dl',{'class':'blind'}).find_all('dd')[4].text.replace(u'\xa0','')
        #s_pr2 = soup.find('dl',{'class':'blind'}).find_all('dd')[5].text.replace(u'\xa0','')
        s_pr3 = soup.find('dl',{'class':'blind'}).find_all('dd')[6].text.replace(u'\xa0','')[3:]
        s_pr4 = soup.find('dl',{'class':'blind'}).find_all('dd')[8].text.replace(u'\xa0','')[3:]
        s_siga = soup.find('div',{'class':'first'}).find('table').find_all('tr')[0].find('td').find('em').text.strip().replace(u'\t','').replace(u'\n','')        
        s_fore = soup.find('div',{'class':'gray'}).find('table').find_all('tr')[2].find('td').find('em').text.strip().replace(u'\t','').replace(u'\n','')        
        s_52h = soup.find('table',{'class':'rwidth'}).find_all('tr')[1].find('td').find_all('em')[0].text
        s_52l = soup.find('table',{'class':'rwidth'}).find_all('tr')[1].find('td').find_all('em')[1].text
        s_target = soup.find('table',{'class':'rwidth'}).find_all('tr')[0].find('td').find_all('em')[1].text
        s_bae = soup.find('table',{'class':'per_table'}).find_all('tr')[3].find('td').find('em').text
        
        #print(s_name, s_price, s_ch, s_ch1, s_ch2)
        #r_data = [s_name, s_price, s_ch, s_ch1, s_ch2, s_pr1, s_pr2, s_pr3, s_pr4]
        r_data = [s_name, s_price, s_ch, s_ch1, s_ch2, s_pr3, s_pr4, s_siga, s_fore, s_52h, s_52l, s_target, s_bae]

        return r_data
    
    except:
        print('no data'+code)

In [ ]:
df_inter = pd.read_csv('interest.csv', encoding="euc-kr")

codes = df_inter['종목코드']

#i = int()
list_realtime = []

for code in codes:
    #print(code[2:])
    #i = i + 1
    #print(i)
    data1 = naver_data(code[2:])
    #print(data1)
    try:
        list_realtime.append(
            {
            '종목명'  :data1[0].replace('\n',''),
            '현재가'  :data1[1],
            '변동금액':int(data1[2].replace(',','')),
            '손익'    :data1[3],
            '비율'    :float(data1[4]),
            '고가'    :data1[5],
            '저가'    :data1[6],
            '시총'    :data1[7],
            '외국인'    :data1[8],
            '52H'    :data1[9],
            '52L'    :data1[10],
            '목표가'    :data1[11],
            '배당률'    :data1[12]                
            })
    
    except:
        print('no data1')

# display(list_realtime)
df_realtime = pd.DataFrame(list_realtime)
df_realtime_sort = df_realtime.sort_values(by=['손익','비율','변동금액'], ascending=False)

display(df_realtime_sort)    


기간별 수익률

In [ ]:
import csv
import requests
import pandas as pd
from bs4 import BeautifulSoup

def fn_data(code):
    url = "http://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?pGB=1&gicode=A"+code+"&cID=&MenuYn=Y&ReportGB=&NewMenuID=11&stkGb=701"
    
    res = requests.get(url)
    res.raise_for_status()
    soup = BeautifulSoup(res.text, 'lxml')    
    
    try:
        s_name = soup.find("h1", {"id":"giName"}).text.replace(u'\xa0','')
        
        #s_code = soup.find("h2").text.replace(u'\xa0','')
        s_code = "e=" + code
        s_cate = soup.find("span",{"class":"stxt1"}).text.replace(u'\xa0',' ')
        s_busi = soup.find("span",{"class":"stxt2"}).text.replace(u'\xa0',' ')
        s_price = soup.find("span",{"id":"svdMainChartTxt11"}).text.replace(u'\xa0','')
        
        s_max_min = soup.find("table",{"class":"us_table_ty1"}).find("tbody").\
                 find("tr",{"class":"zigbg_in"}).find("td",{"class":"r"}).text.replace(u'\xa0','')
        s_mm = s_max_min.split('/')         
        
        #print(s_name)
        
        #기간별 수익률
        s_1m = soup.find("table",{"class":"us_table_ty1"}).find("tbody").\
                 find_all("tr")[2].find("td",{"class":"r"}).find_all("span")[0].text.replace(u'\xa0','')
        s_3m = soup.find("table",{"class":"us_table_ty1"}).find("tbody").\
                 find_all("tr")[2].find("td",{"class":"r"}).find_all("span")[1].text.replace(u'\xa0','')
        s_6m = soup.find("table",{"class":"us_table_ty1"}).find("tbody").\
                 find_all("tr")[2].find("td",{"class":"r"}).find_all("span")[2].text.replace(u'\xa0','')
        s_1y = soup.find("table",{"class":"us_table_ty1"}).find("tbody").\
                 find_all("tr")[2].find("td",{"class":"r"}).find_all("span")[3].text.replace(u'\xa0','')
        s_3y = soup.find("input",{"id":"c3Y"})["value"]
          
        #지표
        s_eps = soup.find_all("table",{"class":"us_table_ty1"})[10].find("tbody").\
                 find_all("tr")[18].find_all("td")[2].text.replace(u'\xa0','')
        s_bps = soup.find_all("table",{"class":"us_table_ty1"})[10].find("tbody").\
                 find_all("tr")[19].find_all("td")[2].text.replace(u'\xa0','')
        s_dps = soup.find_all("table",{"class":"us_table_ty1"})[10].find("tbody").\
                 find_all("tr")[20].find_all("td")[2].text.replace(u'\xa0','')
        
        s_per = soup.find_all("table",{"class":"us_table_ty1"})[10].find("tbody").\
                 find_all("tr")[21].find_all("td")[2].text.replace(u'\xa0','')
        s_pbr = soup.find_all("table",{"class":"us_table_ty1"})[10].find("tbody").\
                 find_all("tr")[22].find_all("td")[2].text.replace(u'\xa0','')
        s_pdr = soup.find_all("table",{"class":"us_table_ty1"})[10].find("tbody").\
                 find_all("tr")[24].find_all("td")[2].text.replace(u'\xa0','')
        
        
        r_data = [s_name, s_code, s_cate, s_busi, s_price,\
                s_1m, s_3m, s_6m, s_1y, s_3y, s_mm[0], s_mm[1], s_eps, s_bps, s_dps, s_per, s_pbr, s_pdr]

        #print(r_data)
        return r_data
    
    except:
        s_name = ""
        s_1m = ""
        s_3m = ""
        s_6m = ""
        s_1y = ""
        s_3y = ""
        s_eps = ""
        s_bps = ""
        s_dbs = ""
        s_per = ""
        s_pbr = ""
        s_pdr = ""
        
        print(code + " error")        


    #print("---------------------------------------")



In [ ]:
import datetime

today = datetime.date.today().strftime('%y%m%d')
filename = today + "_period_profit.csv"
f = open(filename, "w", encoding="euc-kr", newline="")
writer = csv.writer(f)

title = "종목명,종목코드,분류KSE,분류FICS,현재가,1개월,3개월,6개월,1년,3년,최고가,최저가,주당순이익,주당순자산,배당금,PER,PBR,배당률".split(",")
print(type(title))
writer.writerow(title)

file_sise = today + "_sise_code.csv"
df_sise = pd.read_csv(file_sise, encoding="euc-kr")
codes = df_sise['종목코드']

#print(codes)
i = int()

for code in codes:
    #print(code[2:])
    i = i + 1
    if i%100==0 : print(i)
    data = fn_data(code[2:])   
    if data :
        #print(code)
        #print(data)        
        writer.writerow(data)

print('save')
f.close()

분기별 실적 추이

In [5]:
import csv
import requests
import pandas as pd
from bs4 import BeautifulSoup

def fn_qt_data(code):
    #print(code)
    url = "http://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?pGB=1&gicode=A"+code+"&cID=&MenuYn=Y&ReportGB=&NewMenuID=11&stkGb=701"
    #print(url)
    res = requests.get(url)
    res.raise_for_status()
    soup = BeautifulSoup(res.text, 'lxml')
    r_data = list()
    
    try:
        s_name = soup.find("h1", {"id":"giName"}).text.replace(u'\xa0','')
        s_pr = soup.find("span", {"id":"svdMainChartTxt11"}).text.replace(u'\xa0','')
        #print(s_pr) 
        #시총
        #s_total = soup.find("table",{"class":"us_table_ty1"}).find("tbody").\
        #         find_all("tr")[3].find("td",{"class":"r"}).text.replace(u'\xa0','')
        #s_cate = soup.find("span",{"class":"stxt1"}).text.replace(u'\xa0',' ')
        
        #r_data = [s_name, s_total, s_cate]

        for i in range(0,8):
            #r_data = [s_name]
            
            #분기 ---------------------------
            s_q = soup.find_all("table",{"class":"us_table_ty1"})[12].find("thead").\
                     find_all("tr")[1].find_all("th")[i].text.replace(u'\xa0','').strip()[-10:]            
            #print(i,s_q)
            #매출액
            s_mae = soup.find_all("table",{"class":"us_table_ty1"})[12].find("tbody").\
                     find_all("tr")[0].find_all("td")[i].text.replace(u'\xa0','')
            #영업이익
            s_young = soup.find_all("table",{"class":"us_table_ty1"})[12].find("tbody").\
                     find_all("tr")[1].find_all("td")[i].text.replace(u'\xa0','')
            #영업이익률
            #s_yr = soup.find_all("table",{"class":"us_table_ty1"})[12].find("tbody").\
            #         find_all("tr")[14].find_all("td")[i].text.replace(u'\xa0','')
            s_yr = round(int(s_young.replace(',',''))/int(s_mae.replace(',',''))*100,2)
            #print(s_yr)
            
            #당기순이익
            s_dang = soup.find_all("table",{"class":"us_table_ty1"})[12].find("tbody").\
                     find_all("tr")[3].find_all("td")[i].text.replace(u'\xa0','')
            #전년도ROE
            s_roe0 = soup.find_all("table",{"class":"us_table_ty1"})[11].find("tbody").\
                     find_all("tr")[17].find_all("td")[4].text.replace(u'\xa0','')
            #ROE
            s_roe = soup.find_all("table",{"class":"us_table_ty1"})[12].find("tbody").\
                     find_all("tr")[17].find_all("td")[i].text.replace(u'\xa0','')
            #EPS
            s_eps = soup.find_all("table",{"class":"us_table_ty1"})[12].find("tbody").\
                     find_all("tr")[18].find_all("td")[i].text.replace(u'\xa0','')            
            #DPS
            s_dps = soup.find_all("table",{"class":"us_table_ty1"})[12].find("tbody").\
                     find_all("tr")[20].find_all("td")[i].text.replace(u'\xa0','')
            #PBR
            s_pbr = soup.find_all("table",{"class":"us_table_ty1"})[12].find("tbody").\
                     find_all("tr")[22].find_all("td")[i].text.replace(u'\xa0','') 
            
            #BPS
            s_bps = soup.find_all("table",{"class":"us_table_ty1"})[12].find("tbody").\
                     find_all("tr")[19].find_all("td")[i].text.replace(u'\xa0','')
            
            #BPS가 있을경우 계속
            if s_bps:
                #price
                s_price = format(int(int(s_bps.replace(',','')) * float(s_pbr)), ',')
                #print(s_price)
                
                #10년 복리승수
                s_bok = round(1*(1+float(s_roe0)/100)**10, 2)                                    
                #s_bok = 1*(1+0.1)**10
                #print(float(s_roe0)/100)
                #print(s_bok)
                
                #10년후 자산가치
                s_value = int(s_bok * int(s_bps.replace(',',''))) 
                
                #적정가격
                s_tp = format(int(s_value / (1*(1+0.15)**10)), ',')
                
                s_value = format(s_value, ',')
                #print(s_value, s_tp)
                
            else:
                s_price = 0
                s_bok = 0
                s_value = 0
                s_tp = 0
            
            #r_data = r_data + [s_q, s_mae, s_young, s_dang, s_roe, s_eps, s_bps, s_dps, s_pbr]
            r_data = [s_name, s_pr, " "+s_q, s_mae, s_young, s_dang, s_yr, s_roe0, s_roe, s_eps, s_bps, s_dps, s_pbr, s_price, s_bok, s_value, s_tp]
            writer.writerow(r_data)
            #display(r_data)
            
        #return r_data
    
    except:
        s_dang = ""
        s_roe0 = ""
        s_roe = ""
        s_eps = ""
        s_bps = ""
        s_dps = ""
        s_pbr = "" 
        
        print(code)


    #print("---------------------------------------")

In [ ]:
import datetime

today = datetime.date.today().strftime('%y%m%d')
filename = today + "_quarter_record.csv"

f = open(filename, "w", encoding="euc-kr", newline="")
writer = csv.writer(f)

title = "종목명,가격,분기,매출액,영업이익,순이익,영업이익률,전년도ROE,ROE,EPS,BPS,DPS,PBR,분기말종가,복리승수(10),자산가치(10),적정가격".split(",")
writer.writerow(title)

file_sise = today + "_sise_code.csv"
df_sise = pd.read_csv(file_sise, encoding="euc-kr")
codes = df_sise['종목코드']

i = int()

for code in codes:

    fn_qt_data(code[2:])

print('save')
f.close()